In [38]:
import pandas as pd
import numpy as np
import os
import math

In [39]:
### Define wd and file names for reading
wd = 'C:\\Users\\DTRManning\\Desktop\\IndependentProjects\\LearningDP\\BatteryControl'
#fname = 'PriceData.csv'
fname = 'PriceData.csv'

In [40]:
### Combine into absolute file paths
fpath = os.path.join( wd, fname )

In [41]:
### Reference: http://web.mit.edu/15.053/www/AMP-Chapter-11.pdf

In [42]:
### Define battery parameters
maxEnergy = 15
minEnergy = 1

maxCharge    = 5
maxDischarge = 5

# Size of the transition step to discretize SOC states
transitionInterval = 1

In [43]:
### Read in price data. Note the current implementation doesn't automatically add a final period, so this has to be done
### manually in the price file.
priceData = pd.read_csv(fpath)

In [44]:
def DiscretizeSOC( minEnergy, maxEnergy, transitionInterval ):
    
    ### Disretize battery SOC into intervals based on the transition interval size.
    socStates = np.arange(minEnergy, maxEnergy + transitionInterval, transitionInterval) # Add transition interval to include max SOC in states

    ### Create a transition matrix with the accessible states at each SOC. Structure as dictionary of transition vectors for 
    ### fast lookup.
    TransitionVectors = {}

    for curState in socStates:

        # For each state, calculate the distances to see which states are accessible. Battery SOC can't decrease more than 
        # maxDischarge or increase more than maxCharge.
        availableStates = pd.DataFrame( {'AllStates': socStates} )
        availableStates['distance'] = availableStates['AllStates'] - curState
        availableStates = availableStates[(availableStates['distance'] >= -maxDischarge) & (availableStates['distance'] <= maxCharge)]
        availableStates = availableStates['AllStates'].tolist() # Convert to list for easier looping

        TransitionVectors[curState] = availableStates
        
    return(socStates, TransitionVectors)

In [45]:
### Create a dataframe where columns indicate time step and rows indicate state.
### Reduce the available states for battery ramp up at the beginning of the horizon. Max available charge at each timestep
### is timestep * MaxCharge + minSOC
def CreateAvailableStates( priceData, socStates ):
    
    # Dataframes for determining optimal decisions and storing optimal results.
    timesteps = priceData['Time']

    batteryBackwardsPass = pd.DataFrame( 0, index = socStates, columns = timesteps ) # Tracks optimal value
    optimalPath          = pd.DataFrame( 0, index = socStates, columns = timesteps ) # Tracks optimal decision
    
    # Reduce available states at the beginning of the horizon to require starting SOC = min SOC
    for curTimestep in list(batteryBackwardsPass.columns):
    
        curMaxSOC = min( (curTimestep * maxCharge) + minEnergy, maxEnergy )

        for curSOC in socStates:
            if curSOC > curMaxSOC:
                batteryBackwardsPass.loc[curSOC,curTimestep] = math.nan
                
    return(batteryBackwardsPass,optimalPath)

In [46]:
def GetHorizonInfo():
    
    HorizonInfo = {}
    dataLength = len(priceData['Time'])
    HorizonInfo['horizonEnd']   = dataLength - 2 # Subtract one for the last timestep in horizon, one for python indexing
    HorizonInfo['horizonStart'] = 0 - 1 # End on timestep 0, subract one for python indexing
    
    return(HorizonInfo)

In [47]:
### Iterate through each row in each column. For each entry, indicate the lowest cost way of reaching that battery state
def BackwardsPassOpt( batteryBackwardsPass, optimalPath, socStates, TransitionVectors ):
    
    for curTimestep in range(HorizonInfo['horizonEnd'],HorizonInfo['horizonStart'],-1):

        for curSOC in socStates:

            if math.isnan(batteryBackwardsPass.loc[ curSOC, curTimestep ] ):
                continue

            nextSOC = np.array(TransitionVectors[curSOC]) # Use numpy array for faster/easier matrix operations

            curPrice = priceData[priceData['Time'] == curTimestep].iloc[0]['elec_costs']

            ### Calculate the cost/value of the discharge by multiplying possible transitions by the current price
            transitions      = nextSOC - curSOC # Potential transitions. Positive implies charge, negative implies discharge
            transitionValues = transitions * curPrice
            # Combine the values of each transition with the cost associated with the next state.
            totalValues      = np.array(transitionValues + batteryBackwardsPass[curTimestep+1][curSOC + transitions])

            ### Get index of lowest cost/highest value transition
            minCostIndex    = np.argmin(totalValues)
            optimalDecision = transitions[minCostIndex]      # Get the optimal decision
            # Get the cost of the optimal decision. Add to cost of previous timestep SOC.
            minCost         = totalValues[minCostIndex]

            ### Insert optimal values into backwards pass results
            batteryBackwardsPass.loc[ curSOC, curTimestep ] = minCost
            optimalPath.loc[ curSOC, curTimestep ]          = optimalDecision
            
    return( batteryBackwardsPass, optimalPath )

In [48]:
### Forward pass to collect optimal decisions as vector
def ForwardPassOpt(batteryBackwardsPass,optimalPath):
    
    # Populate dictionary with optimal SOC
    OptimalSOCbyTime = {}
    OptimalDispatch = {}

    # Index of the minimum backwards pass value is starting optimal path.
    startSOC = batteryBackwardsPass.loc[ :, 0 ].idxmin()
    OptimalSOCbyTime[list(batteryBackwardsPass.columns)[0]] = startSOC

    # Loop through timesteps, other than the first which is populated in the initialization.
    for curTimestep in list(optimalPath.columns)[:-1]: # Don't include last timestep because loop returns SOC for next timestep

        optimalDispatch = optimalPath.loc[OptimalSOCbyTime[curTimestep],curTimestep]
        OptimalDispatch[curTimestep] = optimalDispatch
        OptimalSOCbyTime[curTimestep+1] =  OptimalSOCbyTime[curTimestep] + optimalDispatch
        
    return(OptimalSOCbyTime,OptimalDispatch)

In [49]:
### Run optimization
socStates, TransitionVectors = DiscretizeSOC( minEnergy, maxEnergy, transitionInterval )
batteryBackwardsPass,optimalPath = CreateAvailableStates( priceData, socStates )
HorizonInfo = GetHorizonInfo()
batteryBackwardsPass, optimalPath = BackwardsPassOpt( batteryBackwardsPass, optimalPath, socStates, TransitionVectors )
OptimalSOCbyTime,OptimalDispatch = ForwardPassOpt(batteryBackwardsPass,optimalPath)

In [50]:
### Return results

In [51]:
batteryBackwardsPass

Time,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
1,-5.7931,-5.7931,-5.7931,-5.7931,-5.7931,-5.7931,-5.7931,-5.7571,-5.6986,-5.6131,...,-1.2525,-0.7335,-0.2345,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
2,NaN,-5.9441,-5.9397,-5.9378,-5.9376,-5.9360,-5.9285,-5.9015,-5.8445,-5.7650,...,-1.6034,-1.0844,-0.5883,-0.3978,-0.3538,-0.2839,-0.2657,-0.2635,-0.2394,0
3,NaN,-6.0951,-6.0863,-6.0825,-6.0821,-6.0789,-6.0639,-6.0459,-5.9904,-5.9169,...,-1.9543,-1.4353,-0.9421,-0.7956,-0.7076,-0.5678,-0.5314,-0.5270,-0.4788,0
4,NaN,-6.2461,-6.2329,-6.2272,-6.2266,-6.2218,-6.1993,-6.1903,-6.1363,-6.0688,...,-2.3052,-1.7862,-1.2959,-1.1934,-1.0614,-0.8517,-0.7971,-0.7905,-0.7182,0
5,NaN,-6.3971,-6.3795,-6.3719,-6.3711,-6.3647,-6.3347,-6.3347,-6.2822,-6.2207,...,-2.6561,-2.1371,-1.6497,-1.5912,-1.4152,-1.1356,-1.0628,-1.0540,-0.9576,0
6,NaN,-6.5481,-6.5261,-6.5166,-6.5156,-6.5076,-6.4701,-6.4686,-6.4266,-6.3666,...,-2.9101,-2.4880,-2.0035,-1.9890,-1.7690,-1.4195,-1.3285,-1.3175,-1.1970,0
7,NaN,NaN,-6.6708,-6.6611,-6.6585,-6.6430,-6.6055,-6.6025,-6.5710,-6.5125,...,-3.1641,-2.8389,-2.3544,-2.3428,-2.0529,-1.6852,-1.5920,-1.5569,-1.1970,0
8,NaN,NaN,-6.8155,-6.8056,-6.8014,-6.7784,-6.7409,-6.7364,-6.7154,-6.6584,...,-3.4181,-3.1898,-2.7053,-2.6966,-2.3368,-1.9509,-1.8555,-1.7963,-1.1970,0
9,NaN,NaN,-6.9602,-6.9501,-6.9443,-6.9138,-6.8763,-6.8703,-6.8598,-6.8043,...,-3.6721,-3.5407,-3.0562,-3.0504,-2.6207,-2.2166,-2.1190,-2.0357,-1.1970,0
10,NaN,NaN,-7.1049,-7.0946,-7.0872,-7.0492,-7.0117,-7.0042,-7.0042,-6.9502,...,-3.9261,-3.8916,-3.4071,-3.4042,-2.9046,-2.4823,-2.3825,-2.2751,-1.1970,0


In [52]:
optimalPath

Time,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
1,0,0,0,0,0,0,4,5,5,5,...,5,5,5,0,0,0,0,0,0,0
2,0,-1,-1,-1,-1,-1,3,5,5,5,...,5,5,5,-1,-1,-1,-1,-1,-1,0
3,0,-2,-2,-2,-2,-2,2,5,5,5,...,5,5,5,-2,-2,-2,-2,-2,-2,0
4,0,-3,-3,-3,-3,-3,1,5,5,5,...,5,5,5,-3,-3,-3,-3,-3,-3,0
5,0,-4,-4,-4,-4,-4,0,5,5,5,...,5,5,5,-4,-4,-4,-4,-4,-4,0
6,0,-5,-5,-5,-5,-5,-1,4,5,5,...,5,5,5,-5,-5,-5,-5,-5,-5,0
7,0,0,-5,-5,-5,-5,-2,3,5,5,...,5,5,4,-5,-5,-5,-5,-5,-5,0
8,0,0,-5,-5,-5,-5,-3,2,5,5,...,5,5,3,-5,-5,-5,-5,-5,-5,0
9,0,0,-5,-5,-5,-5,-4,1,5,5,...,5,5,2,-5,-5,-5,-5,-5,-5,0
10,0,0,-5,-5,-5,-5,-5,0,5,5,...,5,5,1,-5,-5,-5,-5,-5,-5,0


In [53]:
OptimalSOCbyTime

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 5,
 8: 10,
 9: 15,
 10: 15,
 11: 15,
 12: 15,
 13: 10,
 14: 15,
 15: 15,
 16: 15,
 17: 11,
 18: 6,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 5,
 28: 10,
 29: 15,
 30: 15,
 31: 15,
 32: 15,
 33: 15,
 34: 15,
 35: 15,
 36: 15,
 37: 10,
 38: 5,
 39: 5,
 40: 10,
 41: 15,
 42: 11,
 43: 6,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1}

In [54]:
OptimalDispatch

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 4,
 7: 5,
 8: 5,
 9: 0,
 10: 0,
 11: 0,
 12: -5,
 13: 5,
 14: 0,
 15: 0,
 16: -4,
 17: -5,
 18: -5,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 4,
 27: 5,
 28: 5,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: -5,
 37: -5,
 38: 0,
 39: 5,
 40: 5,
 41: -4,
 42: -5,
 43: -5,
 44: 0,
 45: 0,
 46: 0,
 47: 0}